Imports and Loading Model

In [1]:
from sentence_transformers import SentenceTransformer, util
import chromadb

c:\anaconda3\envs\3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = SentenceTransformer("all-MiniLM-L6-v2", device="cpu")

Indexing Data

In [3]:
chroma = chromadb.Client()
# Reset collections
for name in ["docs", "windows"]:
  try: chroma.delete_collection(name) 
  except: pass

# Documents
docs = [
"The Eiffel Tower is in Paris, France. It is one of the most famous landmarks.", 
"The Colosseum is in Rome, Italy. It is an ancient amphitheater.",
"The Statue of Liberty is a gift from France, located in New York, USA.",
"The Burj Khalifa in Dubai is the tallest building in the world."
]

# Parent-doc collection
doc_col = chroma.create_collection("docs")
doc_col.add(
documents=docs,
embeddings=model.encode(docs).tolist(), 
ids=[f"d{i}" for i in range(len(docs))]
)